In [ ]:
import torch
import torchvision

import coremltools as ct

from alphapose.utils.config import update_config
from alphapose.models import builder


cfg_path = "./configs/coco/resnet/256x192_res50_lr1e-3_1x.yaml"
cfg = update_config(cfg_path)

# model_path = "./exp/delme4-256x192_res50_lr1e-3_1x.yaml/model_7.pth"
model_path = "./pretrained_models/fast_res50_256x192.pth"

model = builder.build_sppe(cfg.MODEL, preset_cfg=cfg.DATA_PRESET)
model.load_state_dict(torch.load(model_path), strict=False)

In [ ]:
model.eval()

In [ ]:
# Trace with random data
random_input = torch.rand(1, 3, 256, 192) # after test, will get 'size mismatch' error message with size 256x256
traced_model = torch.jit.trace(model, random_input)

In [ ]:
# pytorch 1.6.0
# Convert to Core ML using the Unified Conversion API
model = ct.convert(
    traced_model,
    inputs=[ct.ImageType(name="input_1", shape=random_input.shape)], #name "input_1" is used in 'quickstart'
)

In [ ]:
# Use PIL to load and resize the image to expected size
from PIL import Image
example_image = Image.open("test_img.png")
print(example_image.size)


# Make a prediction using Core ML
out_dict = model.predict({"input_1": example_image})

# Print out top-1 prediction
print(out_dict["classLabel"])